In [1]:
from useful_scit.imps import (xr, plt, np, pd)
from useful_scit.util.make_folders import make_folders
%load_ext autoreload
%autoreload 2
from sectional_v2.util.Nd.sizedist_class_v2.SizedistributionBins import SizedistributionStationBins
from sectional_v2.util.collocate.collocateLONLAToutput import CollocateLONLATout
from sectional_v2.constants import sized_varListNorESM
#from useful_scit.util import log
import useful_scit.util.log as log
log.ger.setLevel(log.log.INFO)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/mnts/nird/projects//Output_data_OAS-DEV/
/home/ubuntu/mnts/nird/projects/OAS-DEV/sectional_v2/data_info/area_defs.csv


## Savepath:

In [2]:
from sectional_v2.constants import get_outdata_path
path_out = get_outdata_path('eusaar')
version ='_noresmv21_dd'#_noresm2'#_fbvoc'
file_out = path_out + 'Nd_cat_sources_timeseries%s.csv'%version

In [3]:
file_out

'/home/ubuntu/mnts/nird/projects//Output_data_OAS-DEV//eusaar/Nd_cat_sources_timeseries_noresmv21_dd.csv'

### Model data:

In [4]:
nr_of_bins = 5
maxDiameter = 39.6  # 23.6 #e-9
minDiameter = 5.0  # e-9
history_field='.h1.'
#cases_sec = ['SECTv11_ctrl', 'SECTv11_ctrl_fbvoc']
cases_sec = ['SECTv11_ctrl','SECTv11_noresm2_ctrl', 'SECTv11_noresm2_adj']#_fbvoc']
#cases_orig =['noSECTv11_ctrl', 'noSECTv11_ctrl_fbvoc'] #/noSECTv11_ctrl
cases_orig =[ 'noSECTv11_ctrl','noSECTv11_noresm2_ricc']#_fbvoc'] #/noSECTv11_ctrl
#cases_sec = ['SECTv11_noresm2_NFHIST']#'SECTv11_ctrl_fbvoc']#'SECTv11_ctrl']#,'SECTv11_ctrl_fbvoc']#'SECTv11_ctrl']
#cases_orig =['noSECTv11_noresm2_NFHIST']#'noSECTv11_ctrl_fbvoc'] #/no SECTv11_ctrl
cases_sec = ['SECTv21_ctrl','SECTv21_ctrl_koagD']
cases_orig =['noSECTv21_default_dd', 'noSECTv21_ox_ricc_dd']
from_t = '2008-01-01'
to_t = '2010-01-01'



In [5]:
# Make station N50 etc.
if from_t=='2008-01-01' and to_t=='2010-01-01':
    ds_sec={}
    dic_mod_all = {}
    for case_name in cases_sec:
        ls = []
        for f_t, t_t in zip(['2008-01-01','2009-01-01'], ['2009-01-01','2010-01-01']):
            s = SizedistributionStationBins(case_name, f_t, t_t, [minDiameter, maxDiameter], True, 'hour',
                 nr_bins=nr_of_bins, history_field=history_field)
            ls.append(s.return_Nd_ds())
        ds_conc = xr.concat(ls, 'time')
        # remove duplicates in time:
        ds_conc = ds_conc.sel(time=~ds_conc.indexes['time'].duplicated())
        ds_sec[case_name] = ds_conc 
        dic_mod_all[case_name] = ds_sec[case_name]# = s.return_Nd_ds()

    ds_orig={}
    for case_name in cases_orig:
        ls = []
        for f_t, t_t in zip(['2008-01-01','2009-01-01'], ['2009-01-01','2010-01-01']):

            s = SizedistributionStationBins(case_name, f_t, t_t, [minDiameter, maxDiameter], False, 'hour',
                                    nr_bins=nr_of_bins, history_field=history_field)
            ls.append(s.return_Nd_ds())
        ds_conc = xr.concat(ls, 'time')
        # remove duplicates in time:
        ds_conc = ds_conc.sel(time=~ds_conc.indexes['time'].duplicated())
        ds_orig[case_name] = ds_conc 

        #ds_orig[case_name] = xr.concat(ls,'time')
        dic_mod_all[case_name] = ds_orig[case_name]# = s.return_Nd_ds()
    

    
    
else:
    ds_sec={}
    dic_mod_all = {}
    for case_name in cases_sec:
        s = SizedistributionStationBins(case_name, from_t, to_t, [minDiameter, maxDiameter], True, 'hour',
                 nr_bins=nr_of_bins, history_field=history_field)
        ds_sec[case_name] = s.return_Nd_ds()
        dic_mod_all[case_name] = ds_sec[case_name]# = s.return_Nd_ds()

    ds_orig={}
    for case_name in cases_orig:
        s = SizedistributionStationBins(case_name, from_t, to_t, [minDiameter, maxDiameter], False, 'hour',
                                    nr_bins=nr_of_bins, history_field=history_field)
        ds_orig[case_name] = s.return_Nd_ds()
        dic_mod_all[case_name] = ds_orig[case_name]# = s.return_Nd_ds()
    



# Drop excess coordinates in dataset:

In [6]:
def drop_excess_coords(ds):
    coords = list(ds.coords)
    lon_lat_coords = [coo for coo in coords if ('LON' in coo or 'LAT' in coo) or ('lon_' in coo or 'lat_' in coo)]
    return ds.drop(lon_lat_coords)

In [7]:
for case_name in dic_mod_all:
    dic_mod_all[case_name] = drop_excess_coords(dic_mod_all[case_name]).isel(lev=-1).drop(['lev']).sel(time=slice(from_t, to_t))

## Import eusaar

In [8]:
from sectional_v2.constants import path_eusaar_data# path_eusaar_data
import numpy as np
from sectional_v2.util import eusaar_data
from sectional_v2.util.eusaar_data.histc_vars import load_var_as_dtframe
from sectional_v2.util.eusaar_data import  distc_var, histc_vars, histsc_hists # import load_var_as_dtframe
import matplotlib.pyplot as plt
from useful_scit.plot import get_cmap_dic

### Rename N30 to N30-50

In [9]:
st_ds = histc_vars.get_histc_vars_xr()
st_ds = st_ds.rename_vars({'N30':'N30-50'})

### Make vars with source name:

In [10]:
for case_name in dic_mod_all:
    dic_mod_all[case_name]['N50-100']=dic_mod_all[case_name]['N50']-dic_mod_all[case_name]['N100']
    dic_mod_all[case_name]['N30-100']=dic_mod_all[case_name]['N30-50']+dic_mod_all[case_name]['N50-100']
st_ds['N50-100']=st_ds['N50']-st_ds['N100']
st_ds['N30-100']=st_ds['N30-50']+st_ds['N50-100']

## Add flags where observation data good:

In [11]:
st_ds['time']

<xarray.DataArray 'time' (time: 17545)>
array(['2008-01-01T00:00:00.000000000', '2008-01-01T01:00:00.000000000',
       '2008-01-01T02:00:00.000000000', ..., '2009-12-31T22:00:00.000000000',
       '2009-12-31T23:00:00.000000000', '2010-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01 2008-01-01T01:00:00 ... 2010-01-01

In [12]:
from sectional_v2.util.eusaar_data.flags import load_flags_allstations
flags = load_flags_allstations()

In [13]:
for case_name in dic_mod_all:
    dic_mod_all[case_name] = dic_mod_all[case_name].sel(time=slice(flags['time'].min(),flags['time'].max())) 
    dic_mod_all[case_name]['flag_gd'] = flags['gd']
    
st_ds['flag_gd'] = flags['gd']

## Combine to one dataset:

In [14]:
dic_all_df = {}
first=True
# Merge model cases
for case_name in dic_mod_all:
    dic_all_df[case_name] = dic_mod_all[case_name].to_dataframe()
    dic_all_df[case_name]['source']=case_name    
    if first:
        df_mod_time = dic_all_df[case_name].reset_index()
        first=False
    else:
        df_mod_time = pd.merge(df_mod_time, dic_all_df[case_name].reset_index(), how='outer')
# set source in observations
df_obs_time =  st_ds.to_dataframe()
df_obs_time['source']='eusaar'

In [15]:
df_merged = pd.merge(df_obs_time.reset_index(), df_mod_time, how='outer')
df_merged

,station,time,N30-50,N50,N100,N250,N50-100,N30-100,flag_gd,source
0,KPO,2008-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
1,KPO,2008-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
2,KPO,2008-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
3,KPO,2008-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
4,KPO,2008-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
...,...,...,...,...,...,...,...,...,...,...
2101987,CMN,2009-12-31 20:00:00,560.563786,1363.795515,510.999761,43.245130,852.795755,1413.359541,False,noSECTv21_ox_ricc_dd
2101988,CMN,2009-12-31 21:00:00,580.296219,1412.597669,528.737033,44.528629,883.860635,1464.156854,False,noSECTv21_ox_ricc_dd
2101989,CMN,2009-12-31 22:00:00,509.633197,1244.595527,465.639294,38.915290,778.956233,1288.589430,False,noSECTv21_ox_ricc_dd
2101990,CMN,2009-12-31 23:00:00,464.145948,1139.192896,426.463424,35.432914,712.729472,1176.875421,False,noSECTv21_ox_ricc_dd


## Construnc N30-100 and N50-100
Mark that N50=N50-500 and N100=N100-500, so 
$$N_{50-100}=N_{50}-N_{100}$$

In [16]:
df_merged['N30-100'] = df_merged['N30-50']+ df_merged['N50']-df_merged['N100']
df_merged['N50-100'] =df_merged['N50']-df_merged['N100']

In [17]:
from useful_scit.util.make_folders import make_folders
make_folders(file_out)
df_merged.to_csv(file_out)

In [18]:
df_merged

,station,time,N30-50,N50,N100,N250,N50-100,N30-100,flag_gd,source
0,KPO,2008-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
1,KPO,2008-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
2,KPO,2008-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
3,KPO,2008-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
4,KPO,2008-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
...,...,...,...,...,...,...,...,...,...,...
2101987,CMN,2009-12-31 20:00:00,560.563786,1363.795515,510.999761,43.245130,852.795755,1413.359541,False,noSECTv21_ox_ricc_dd
2101988,CMN,2009-12-31 21:00:00,580.296219,1412.597669,528.737033,44.528629,883.860635,1464.156854,False,noSECTv21_ox_ricc_dd
2101989,CMN,2009-12-31 22:00:00,509.633197,1244.595527,465.639294,38.915290,778.956233,1288.589430,False,noSECTv21_ox_ricc_dd
2101990,CMN,2009-12-31 23:00:00,464.145948,1139.192896,426.463424,35.432914,712.729472,1176.875421,False,noSECTv21_ox_ricc_dd


In [19]:
df_merged['source'].unique()

array(['eusaar', 'SECTv21_ctrl', 'SECTv21_ctrl_koagD',
       'noSECTv21_default_dd', 'noSECTv21_ox_ricc_dd'], dtype=object)

In [20]:
df_merged#[df_merged['source']=='SECTv11_ctrl_fbvoc']#.unique()

,station,time,N30-50,N50,N100,N250,N50-100,N30-100,flag_gd,source
0,KPO,2008-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
1,KPO,2008-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
2,KPO,2008-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
3,KPO,2008-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
4,KPO,2008-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,False,eusaar
...,...,...,...,...,...,...,...,...,...,...
2101987,CMN,2009-12-31 20:00:00,560.563786,1363.795515,510.999761,43.245130,852.795755,1413.359541,False,noSECTv21_ox_ricc_dd
2101988,CMN,2009-12-31 21:00:00,580.296219,1412.597669,528.737033,44.528629,883.860635,1464.156854,False,noSECTv21_ox_ricc_dd
2101989,CMN,2009-12-31 22:00:00,509.633197,1244.595527,465.639294,38.915290,778.956233,1288.589430,False,noSECTv21_ox_ricc_dd
2101990,CMN,2009-12-31 23:00:00,464.145948,1139.192896,426.463424,35.432914,712.729472,1176.875421,False,noSECTv21_ox_ricc_dd
